In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweetdata/labeled_data.csv


In [3]:
tweets = pd.read_csv("../input/tweetdata/labeled_data.csv")

In [4]:
tweets.head

<bound method NDFrame.head of        Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                             

In [65]:
text_data = tweets["tweet"]
hatespeech_data = (tweets[tweets["class"] == 0])["tweet"]
offensivelanguage_data = (tweets[tweets["class"] == 1])["tweet"]
neither_data = (tweets[tweets["class"] == 2])["tweet"]

classes = [hatespeech_data, offensivelanguage_data, neither_data]
names = ["hate speech", "offensive language", "neither"]

#Majority have been marked offensive language, followed by neither (less than a quarter), and hatespeech
for i in range(len(classes)):
    print(names[i], len(classes[i]))

hate speech 1430
offensive language 19190
neither 4163


In [61]:
top20s = []
i = 0
for dataset in classes:
    text = ""
    for line in dataset:
        text += line + " "
    # print(text)
    def checktoinclude(word):
        for c in word:
            if c in string.punctuation or c in "0123456789":
                return False
        return True

    vocabulary = {}
    vocab = [word.lower() for word in text.split() if (word not in string.punctuation and checktoinclude(word))]
    for word in vocab:
        if word in vocabulary.keys():
            vocabulary[word]+= 1
        else:
            vocabulary[word] = 1
    sorted_vocab = (sorted(((value,key) for (key,value) in vocabulary.items()), reverse=True))

    # Remove stop words
    stop_word_list = ["get", "got", "like", "all", "just", "what", "these", "the", "you", "my", "me", "your", 
                      "if", "u", "ur", "do", "we", "a", "to", "how", "know", "knew", 
                      "of", "in", "and", "or", "on", "for", "this", "that", "go", "want", "why",
                      "with", "as", "at", "its", "by", "is", "an", "has", "it", "i",
                      "was", "are", "be", "but", "us", "not", "they", "then", "than", "were",
                      "said", "from", "his", "after", "will", "have", "their", "over",
                      "more", "he", "she", "up", "into", "out", "who", "when", "been","had", 
                      "about", "which", "could", "can", "some", "so", "here", "there", "him", "her", "them", "yes", "no"]
    # print(sorted_vocab[:200])
    # print([v for (k,v) in sorted_vocab])
    new_vocab = [v for (k,v) in sorted_vocab if v not in stop_word_list]
    # new_vocab = [s for s in new_vocab if s not in stop_word_list]
    top20 = new_vocab[:20]
    print(names[i], ':\nTop 20 most frequent words: ', top20, '\n')
    for wd in top20:
        print(wd, ':', vocabulary[wd])
    print('--------------------------------------------\n')
    top20s.append(top20)
    i+= 1

hate speech :
Top 20 most frequent words:  ['rt', 'bitch', 'faggot', 'ass', 'white', 'fuck', 'nigga', 'fucking', 'nigger', 'trash', 'niggas', 'hate', 'fag', 'bitches', 'shit', 'hoes', 'people', 'lol', 'pussy', 'faggots'] 

rt : 322
bitch : 171
faggot : 149
ass : 128
white : 123
fuck : 115
nigga : 114
fucking : 93
nigger : 83
trash : 71
niggas : 67
hate : 57
fag : 57
bitches : 54
shit : 51
hoes : 45
people : 42
lol : 40
pussy : 38
faggots : 38
--------------------------------------------

offensive language :
Top 20 most frequent words:  ['bitch', 'rt', 'bitches', 'hoes', 'pussy', 'hoe', 'ass', 'fuck', 'shit', 'nigga', 'lol', 'niggas', 'love', 'yo', 'fucking', 'one', 'bad', 'ya', 'im', 'good'] 

bitch : 6568
rt : 5936
bitches : 2623
hoes : 1922
pussy : 1687
hoe : 1425
ass : 1342
fuck : 1196
shit : 993
nigga : 933
lol : 769
niggas : 638
love : 556
yo : 539
fucking : 524
one : 491
bad : 472
ya : 429
im : 388
good : 381
--------------------------------------------

neither :
Top 20 most fr

In [62]:
# Check commonalities of most frequent words

for i in range(3):
    for j in range(i+1, 3):
            print("Common words in top20 between", names[i], 'and', names[j], ':')
            print(set(top20s[i])&set(top20s[j]), ', ', len(set(top20s[i])&set(top20s[j])), 'words\n')

Common words in top20 between hate speech and offensive language :
{'niggas', 'nigga', 'shit', 'bitch', 'rt', 'fucking', 'ass', 'lol', 'pussy', 'bitches', 'fuck', 'hoes'} ,  12 words

Common words in top20 between hate speech and neither :
{'trash', 'lol', 'rt'} ,  3 words

Common words in top20 between offensive language and neither :
{'good', 'rt', 'one', 'love', 'lol'} ,  5 words

